In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import os
import json
import openai
import pandas as pd
from datasets import load_dataset

Импорт датасета и созадние jsonl файла для дообучения

In [3]:
dataset = load_dataset("inkoziev/jokes_dialogues")

jokes_df = dataset["train"].to_pandas()

sampled_jokes = jokes_df.sample(n=50)

chat_data = []
for _, record in sampled_jokes.iterrows():
    chat_data.append({
        "messages": [
            {"role": "system", "content": "You are a humorous assistant."},
            {"role": "user", "content": record['context']},
            {"role": "assistant", "content": record['utterance']}
        ]
    })

output_file = "jokes.jsonl"
with open(output_file, "w", encoding="utf-8") as file:
    for chat_item in chat_data:
        file.write(json.dumps(chat_item, ensure_ascii=False) + "\n")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

jokes_dialog_dataset.json:   0%|          | 0.00/61.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/114579 [00:00<?, ? examples/s]

Загрузка данных в openai

In [4]:
!curl https://api.openai.com/v1/files \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -F purpose="fine-tune" \
  -F file="@/content/jokes.jsonl"


{
  "object": "file",
  "id": "file-KsZz46M23UG9CteZLBig7A",
  "purpose": "fine-tune",
  "filename": "jokes.jsonl",
  "bytes": 26580,
  "created_at": 1734778541,
  "status": "processed",
  "status_details": null
}


Дообучение модели

In [5]:
client = openai.OpenAI()

client.fine_tuning.jobs.create(
    training_file="file-KsZz46M23UG9CteZLBig7A",
    model="gpt-4o-mini-2024-07-18"
)

FineTuningJob(id='ftjob-MR9kJoTUjES31PlezHIPPQIa', created_at=1734778569, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-yyyPGCw9VCJjgaycEyhn8xFf', result_files=[], seed=1831897372, status='validating_files', trained_tokens=None, training_file='file-KsZz46M23UG9CteZLBig7A', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'batch_size': 'auto', 'learning_rate_multiplier': 'auto', 'n_epochs': 'auto'}}})

In [12]:
client.fine_tuning.jobs.retrieve("ftjob-MR9kJoTUjES31PlezHIPPQIa")

FineTuningJob(id='ftjob-MR9kJoTUjES31PlezHIPPQIa', created_at=1734778569, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::AgreyVd1', finished_at=1734779406, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-yyyPGCw9VCJjgaycEyhn8xFf', result_files=['file-KaY7GZgaHxkBYkESGC1zUa'], seed=1831897372, status='succeeded', trained_tokens=13581, training_file='file-KsZz46M23UG9CteZLBig7A', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'n_epochs': 3, 'batch_size': 1, 'learning_rate_multiplier': 1.8}}})